In [1]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
#display(HTML("<style>.container { font-weight: bold !important; }</style>"))
#display(HTML("<style>.container { width: 98% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

# 교차 검증

- 고정된 학습과 테스트 데이터로 평가를 하다보면 테스트 데이터에만 최적의 성능을 낸다. 
- 데이타 편중 및 과적합(Overfitting) 문제점
- 본고사를 치르기전에 모의 고사를 여러번 보는 것과 같은 효과 
- K 폴드, Stratified 폴드
- cross_val_score( )



## K 폴드 
- 가장 보편적으로 사용되는 교차 검증 기법 
- 먼저 K개의 데이터 폴드 세트를 만들어서 K번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행
``` 
 - 폴더 세트 설정 
 - for 루프에서 반복적으로 학습과 테스트 데이터의 인덱스 추출 
 - 반복적으로 학습과 예측을 수행하고 예측 성능을 반환한다
```


#### 수행 과정 

```
(1) 데이타와 라벨로 분리 

(2) 모델 생성 

(3) kfold 객체 생성 
n 은 폴더 갯수 
kfold = KFold(n_splits=n)

(4) KFold객체의 split( ) 호출 + for 반복문 
# 정확도 리스트 
cv_accuracy = []

# 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 모델 학습 및 예측 
    예측기.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정해서 정확도 리스트에 추가 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    cv_accuracy.append(accuracy)

# 평균 정확도는?
np.mean(cv_accuracy)

```


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

###  5개로 kfold를 지정해서 모델 생성 후 테스트 

In [4]:
from sklearn.model_selection import KFold

In [6]:
print(dir(KFold))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_iter_test_indices', '_iter_test_masks', 'get_n_splits', 'split']


In [5]:
#데이터 준비
from sklearn.datasets import load_iris

iris = load_iris()
#X-data
features = iris.data
#y-target
label = iris.target

In [7]:
# model_df = DecisionTreeClassifier(random_state=11)
model_kn = KNeighborsClassifier()

In [8]:
#kfold(n_splits=?)
#? 갯수만큼 서브폴더 생성
kfold = KFold(n_splits=5)

#정확도 리스트
cv_accuracy = []

#횟수 저장용 변수 초기화
n_iter = 0

#kfold.split(X_data)
for train_index, test_index  in kfold.split(features):
    #학습용, 테스트 데이터 분리
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    #모델 학습
    model_kn.fit(X_train, y_train)    
    pred = model_kn.predict(X_test)
    n_iter += 1
    
    #정확도 구한 후 리스트에 삽입
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    cv_accuracy.append(accuracy)
    
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    
    

print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 :0.8333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 :0.8, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.91332


# Stratified K 폴드

- 데이타의 분포도가 유사해야한다는 kFold의 특징을 보완 
- 불균형한 분포도를 가진 레이블 데이타 집합을 위한 k폴드 방식 
- 불균형한 분포도를 가진 레이블 데이터 집합은 특정 레이블 값이 특이하게 많거나 매우 적어서 레이블의 값의 분포가 한쪽으로 치우친다.
- 예) 대출사기 데이타. 대출사기 레이블이 전체의 0.0001% 에 밖에 해당하지 못한다.  
- 분류 모델에는 적합하나 회귀에서는 연속된 숫자값이기 때문에 적합하지 못하다.


#### 수행 과정 

from sklearn.model_selection import StratifiedKFold

```
(1) 데이타와 라벨로 분리 

(2) 모델 생성 

(3) StratifiedKFold 객체 생성 

n 은 폴더 갯수 
skf = StratifiedKFold(n_splits=n)

(4) StratifiedKFold의 split( ) 호출 + for 반복문 
# 정확도 리스트 
cv_accuracy = []

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 모델 학습 및 예측 
    예측기.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    
    # 반복 시 마다 정확도 측정해서 정확도 리스트에 추가 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    cv_accuracy.append(accuracy)

# 평균 정확도는?
np.mean(cv_accuracy)

```

###  kfold의 문제점 파악하기 

In [10]:
#X-data
features = iris.data
#y-target
#label = iris.target

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target

In [11]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [13]:
iris_df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'label'],
      dtype='object')

In [14]:
kfold = KFold(n_splits=5)

n_iter =0
for train_index, test_index  in kfold.split(iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)']]):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('\n## 교차 검증: {0}'.format(n_iter))
    print('\n학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('\n검증 레이블 데이터 분포:\n', label_test.value_counts())
    print('='*50)
    


## 교차 검증: 1

학습 레이블 데이터 분포:
 1    50
2    50
0    20
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    30
Name: label, dtype: int64

## 교차 검증: 2

학습 레이블 데이터 분포:
 2    50
1    40
0    30
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    20
1    10
Name: label, dtype: int64

## 교차 검증: 3

학습 레이블 데이터 분포:
 0    50
2    50
1    20
Name: label, dtype: int64

검증 레이블 데이터 분포:
 1    30
Name: label, dtype: int64

## 교차 검증: 4

학습 레이블 데이터 분포:
 0    50
1    40
2    30
Name: label, dtype: int64

검증 레이블 데이터 분포:
 2    20
1    10
Name: label, dtype: int64

## 교차 검증: 5

학습 레이블 데이터 분포:
 0    50
1    50
2    20
Name: label, dtype: int64

검증 레이블 데이터 분포:
 2    30
Name: label, dtype: int64


### StratifiedKFold 활용 => 레벨의 분포를 고르게 한다 

In [15]:
from sklearn.model_selection import StratifiedKFold

In [16]:
#객체 생성
skf = StratifiedKFold(n_splits=5)
n_iter=0

#y값도 넣어야
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('\n## 교차 검증: {0}'.format(n_iter))
    print('\n학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('\n검증 레이블 데이터 분포:\n', label_test.value_counts())
    print('='*50)


## 교차 검증: 1

학습 레이블 데이터 분포:
 0    40
1    40
2    40
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    10
1    10
2    10
Name: label, dtype: int64

## 교차 검증: 2

학습 레이블 데이터 분포:
 0    40
1    40
2    40
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    10
1    10
2    10
Name: label, dtype: int64

## 교차 검증: 3

학습 레이블 데이터 분포:
 0    40
1    40
2    40
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    10
1    10
2    10
Name: label, dtype: int64

## 교차 검증: 4

학습 레이블 데이터 분포:
 0    40
1    40
2    40
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    10
1    10
2    10
Name: label, dtype: int64

## 교차 검증: 5

학습 레이블 데이터 분포:
 0    40
1    40
2    40
Name: label, dtype: int64

검증 레이블 데이터 분포:
 0    10
1    10
2    10
Name: label, dtype: int64


##### Stratified 폴더를 이용한 모델 생성과 정확도 분석 

In [17]:
model_kn =  KNeighborsClassifier()

skfold = StratifiedKFold(n_splits=5)
n_iter=0
cv_accuracy=[]

for train_index, test_index  in skfold.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    model_kn.fit(X_train , y_train)    
    pred = model_kn.predict(X_test)

    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)

print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.mean(cv_accuracy)) 


#1 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#1 검증 세트 인덱스:[  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]

#2 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#2 검증 세트 인덱스:[ 10  11  12  13  14  15  16  17  18  19  60  61  62  63  64  65  66  67
  68  69 110 111 112 113 114 115 116 117 118 119]

#3 교차 검증 정확도 :0.9333, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#3 검증 세트 인덱스:[ 20  21  22  23  24  25  26  27  28  29  70  71  72  73  74  75  76  77
  78  79 120 121 122 123 124 125 126 127 128 129]

#4 교차 검증 정확도 :0.9667, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#4 검증 세트 인덱스:[ 30  31  32  33  34  35  36  37  38  39  80  81  82  83  84  85  86  87
  88  89 130 131 132 133 134 135 136 137 138 139]

#5 교차 검증 정확도 :1.0, 학습 데이터 크기: 120, 검증 데이터 크기: 30
#5 검증 세트 인덱스:[ 40  41  42  43  44  45  46  47  48  49  90  91  92  93  94  95  96  97
  98  99 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도: [0.9667 1.     0.9333 0.9667 1.    ]
## 평균 검증

# cross_val_score( )

- 사이킷런에서 제공하는 교차 검증을 편리하게 수행할 수 있게 도와주는 API 
```
cross_val_score(모델 , X_data , y , scoring='accuracy',  cv=폴더갯수)
```



In [18]:
from sklearn.model_selection import cross_val_score 

In [19]:
#데이터 준비
iris_data = load_iris()
model_kn =  KNeighborsClassifier()

#X
data = iris_data.data
#y
label = iris_data.target

scores = cross_val_score(model_kn , data , label , scoring='accuracy', cv=5)

print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))


교차 검증별 정확도: [0.9667 1.     0.9333 0.9667 1.    ]
평균 검증 정확도: 0.9733


# GridSearchCV


```
(1) 딕셔너리 형태로 파라미터를 지정 
parameters = {'max_depth':[1, 2, 3], 'min_samples_split':[2,3]}

(2) GridSearchCV 메서드 적용 
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True, return_train_score=True)

```

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [7]:
from sklearn.model_selection import GridSearchCV, train_test_split

In [8]:
#데이터셋 준비
from sklearn.datasets import load_iris

iris_data = load_iris()
#X
data = iris_data.data
#y
label = iris_data.target

#학습 / 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=11)

In [9]:
#모델 적용
model_kn = KNeighborsClassifier()

#모델에 적용할 하이퍼파라미터값 지정
#parameters = {'하이퍼파라미터변수' : [값1, 값2, ...]}
parameters = {'n_neighbors':list(range(3,10,2))}

#그리드서치 객체 생성
#GridSearchCV(모델명, param_grid=하이퍼파라미터 딕셔너리, cv=폴더갯수, 
#          (옵션)             return_train_score=True, refit=True)
model_kn_grid = GridSearchCV(model_kn, param_grid=parameters, cv=5, 
                             return_train_score=True, refit=True)

#그리드서치에 데이터 적용
model_kn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, return_train_score=True)

In [10]:
print(dir(model_kn_grid))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_is_fitted', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_pairwise', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_validate_data', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_jobs', 'n_splits_', 'param_grid', 'pre_dispatch', 'predict', 'predict_log_pro

In [11]:
#그리드서치객체명.best_params_ : 점수를 최대한 높게 낼 수 있는 하이퍼파라미터 최적값
#그리드서치객체명.best_score_ : 최고점수
print('GridSearchCV 최적 파라미터:', model_kn_grid.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(model_kn_grid.best_score_))

GridSearchCV 최적 파라미터: {'n_neighbors': 5}
GridSearchCV 최고 정확도: 0.9583


In [14]:
#각각의 훈련세트 점수
#딕셔너리 {키 : 값 리스트 ...}
#model_kn_grid.cv_results_

In [12]:
#각각의 훈련세트 점수 -> 데이터프레임
scores_df = pd.DataFrame(model_kn_grid.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.000399,0.000488,0.001393,0.000485,3,{'n_neighbors': 3},0.916667,0.875000,0.958333,1.000000,...,0.950000,0.048591,2,0.968750,0.979167,0.968750,0.968750,0.958333,0.968750,0.006588
1,0.000000,0.000000,0.000000,0.000000,5,{'n_neighbors': 5},0.916667,0.958333,0.958333,0.958333,...,0.958333,0.026352,1,0.968750,0.979167,0.979167,0.979167,0.958333,0.972917,0.008333
2,0.000000,0.000000,0.000000,0.000000,7,{'n_neighbors': 7},0.875000,0.958333,0.958333,0.958333,...,0.950000,0.040825,2,0.979167,0.968750,0.968750,0.979167,0.968750,0.972917,0.005103
3,0.000000,0.000000,0.003125,0.006250,9,{'n_neighbors': 9},0.875000,0.958333,1.000000,0.916667,...,0.950000,0.048591,2,0.979167,0.979167,0.968750,0.979167,0.968750,0.975000,0.005103


In [15]:
scores_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score',
       'split2_test_score', 'split3_test_score', 'split4_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'split3_train_score', 'split4_train_score', 'mean_train_score',
       'std_train_score'],
      dtype='object')

In [17]:
scores_df[['rank_test_score','param_n_neighbors','mean_test_score']]

,rank_test_score,param_n_neighbors,mean_test_score
0,2,3,0.950000
1,1,5,0.958333
2,2,7,0.950000
3,2,9,0.950000


In [18]:
print(f'GridSearchCV 최적 파라미터: {model_kn_grid.best_params_}' )
print(f'GridSearchCV 최고 정확도: {model_kn_grid.best_score_}')

GridSearchCV 최적 파라미터: {'n_neighbors': 5}
GridSearchCV 최고 정확도: 0.9583333333333334


# 퀴즈

- 1) 'titanic_book.csv' 타이타닉 데이타셋을 아래와 같은 전처리 후 cross_val_score() 메서드를 이용하여 교차 검증한 후 
<br> 평균 정확도를 구하여라. 모델의 알고리즘은 임의선정, cv는 5 

cross_val_score(model_kn, X_titanic_df , y_titanic_df , cv=5)


- 2) 1)의 데이타셋을 활용하여 GridSearchCV의 최적 하이퍼 파라미터를 찾은 후 정확도를 구하여라. 

parameters = {'n_neighbors':list(range(3,10))}

GridSearchCV(model_kn , param_grid=parameters , scoring='accuracy' , cv=5)

In [19]:
from sklearn.preprocessing import LabelEncoder

titanic_df = pd.read_csv('titanic_book.csv')


titanic_df['Age'].fillna(titanic_df['Age'].mean(),inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)


encoder = LabelEncoder()
encoder.fit(titanic_df['Sex'])
temp = encoder.transform(titanic_df['Sex'])
titanic_df['Sex'] = temp.reshape(-1, 1)

encoder.fit(titanic_df['Embarked'])
temp = encoder.transform(titanic_df['Embarked'])
titanic_df['Embarked'] = temp.reshape(-1, 1)

titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
encoder.fit(titanic_df['Cabin'])
temp = encoder.transform(titanic_df['Cabin'])
titanic_df['Cabin'] = temp.reshape(-1, 1)


titanic_df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

## 1

In [25]:
from sklearn.model_selection import cross_val_score

X_titanic_df = titanic_df.loc[:, 'Pclass':]
y_titanic_df = titanic_df['Survived']

model_kn = KNeighborsClassifier()
# model_lr = LogisticRegression()
# model_dt = DecisionTreeClassifier(random_state=11)
# model_rf = RandomForestClassifier(random_state=11)

scores = cross_val_score(model_kn, X_titanic_df , y_titanic_df , cv=5)

for iter_count, accuracy in enumerate(scores):
    print(f'교차 검증 {iter_count} 정확도: {accuracy:.4f}')

print(f'\n평균 정확도: {np.mean(scores):.4f}')

교차 검증 0 정확도: 0.6760
교차 검증 1 정확도: 0.6742
교차 검증 2 정확도: 0.6966
교차 검증 3 정확도: 0.7472
교차 검증 4 정확도: 0.7303

평균 정확도: 0.7049


In [26]:
scores

array([0.67597765, 0.6741573 , 0.69662921, 0.74719101, 0.73033708])

In [27]:
print(dir(cross_val_score))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__wrapped__']


## 2

In [28]:
print(dir(model_kn_grid))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_is_fitted', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_pairwise', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_validate_data', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_jobs', 'n_splits_', 'param_grid', 'pre_dispatch', 'predict', 'predict_log_pro

In [29]:
# 0) 학습 + 테스트 분리 
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, 
                                                    test_size=0.2, random_state=11,
                                                    stratify=y_titanic_df)

# 1) 모델 생성 
model_kn = KNeighborsClassifier()

# 2) 모델에 적용할 하이퍼파라미터 값 지정 
# parameters = { '하이퍼파라미터변수':[값1, 값2 .... ]}
parameters = {'n_neighbors':list(range(3,10,2))}
# [3, 5, 7, 9]

# 3) 그리드서치 객체 생성 
# GridSearchCV(모델명, param_grid=하이퍼파라미터딕셔너리, cv=폴더갯수, 
#               return_train_score=True, refit=True)
model_kn_grid = GridSearchCV(model_kn, param_grid=parameters, cv=5, \
                             return_train_score=True, verbose=1)
#  refit=True , return_train_score = False 디폴트 

# 4) 그리드서치에 데이타 적용 
model_kn_grid.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, return_train_score=True,
             verbose=1)

In [30]:
# 그리드서치객체명.best_params_ : 하이퍼파라미터 최적값 
# 그리드서치객체명.best_score_ : 하이퍼파라미터 최적으로 적용했을때의 최고점수 
print('GridSearchCV 최적 파라미터:', model_kn_grid.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(model_kn_grid.best_score_))

GridSearchCV 최적 파라미터: {'n_neighbors': 5}
GridSearchCV 최고 정확도: 0.7346


In [31]:
# 각각의 훈련세트 점수 => 데이타프레임 
scores_df = pd.DataFrame(model_kn_grid.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.004122,5.865778e-03,0.001996,0.002525,3,{'n_neighbors': 3},0.713287,0.622378,0.697183,0.753521,...,0.706569,0.046944,3,0.840070,0.848858,0.849123,0.833333,0.835088,0.841294,0.006662
1,0.006845,7.248190e-03,0.003922,0.006050,5,{'n_neighbors': 5},0.741259,0.685315,0.746479,0.767606,...,0.734610,0.027235,1,0.799649,0.824253,0.780702,0.787719,0.800000,0.798465,0.014831
2,0.002194,3.989697e-04,0.005385,0.000798,7,{'n_neighbors': 7},0.664336,0.678322,0.753521,0.767606,...,0.716419,0.040409,2,0.789104,0.804921,0.763158,0.775439,0.778947,0.782314,0.014020
3,0.002992,9.536743e-08,0.004188,0.000399,9,{'n_neighbors': 9},0.671329,0.671329,0.697183,0.739437,...,0.695292,0.024917,4,0.775044,0.789104,0.763158,0.763158,0.756140,0.769321,0.011609


In [32]:
scores_df[['rank_test_score','param_n_neighbors', 'mean_test_score']]

,rank_test_score,param_n_neighbors,mean_test_score
0,3,3,0.706569
1,1,5,0.734610
2,2,7,0.716419
3,4,9,0.695292


In [33]:
print(f'GridSearchCV 최적 파라미터: {model_kn_grid.best_params_}' )
print(f'GridSearchCV 최고 정확도: {model_kn_grid.best_score_}')

GridSearchCV 최적 파라미터: {'n_neighbors': 5}
GridSearchCV 최고 정확도: 0.7346104599625727


In [34]:
model_kn_grid.best_estimator_

KNeighborsClassifier()

In [35]:
best_model_kn = model_kn_grid.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행. 
pred = best_model_kn.predict(X_test)
accuracy = accuracy_score(y_test , pred)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy))

테스트 세트에서의 DecisionTreeClassifier 정확도 : 0.7318
